<a href="https://colab.research.google.com/github/sddavicillo/TRANSCEFR/blob/main/deBERTa_TClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis, preprocesamiento y preparación de los datos

In [ ]:
# Instalar dependencias (solo necesario en la primera ejecución)
!pip install -U -q PyDrive

# Importar bibliotecas necesarias
import os
import pandas as pd
import re

from google.colab import drive
drive.mount('/content/drive')

# Ruta a tu carpeta con los archivos txt
folder_path = '/content/drive/My Drive/raw_data_CEFR-Predict/ICNALE/SM_0_Unclassified_Unmerged'  # Cambiar por tu ruta

# Lista para almacenar los datos
data = []

# Expresión regular para extraer las etiquetas
pattern = re.compile(r'(A2|B1_1|B1_2|B2)')

# Procesar cada archivo en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        # Construir ruta completa
        file_path = os.path.join(folder_path, filename)

        # Leer contenido del archivo
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            content = file.read()

        # Extraer etiqueta del nombre de archivo
        match = pattern.search(filename)
        if match:
            label = match.group()
        else:
            label = 'Desconocido'  # Para manejar posibles errores

        # Añadir a la lista de datos
        data.append({
            'content': content,
            'filename': filename,
            'label': label
        })

# Crear DataFrame
df = pd.DataFrame(data)

# Mostrar las primeras filas del dataframe
print(df.head())

Mounted at /content/drive
                                             content  \
0  \n\nI think it is not imp – not always importa...   
1  \n\nI agree with it.  In my opinion part-time ...   
2  \n\nI agree the opinion because I in my – in m...   
3  \n\nI can see the benefit of part-time job, bu...   
4  \n\nI disagree with that statement because doi...   

                   filename label  
0  SM_JPN_PTJ1_102_B1_2.txt  B1_2  
1  SM_JPN_PTJ1_103_B1_2.txt  B1_2  
2  SM_JPN_PTJ2_109_B1_2.txt  B1_2  
3  SM_JPN_PTJ1_104_B1_2.txt  B1_2  
4  SM_JPN_PTJ2_104_B1_2.txt  B1_2  


In [ ]:
# Opcional: Guardar el dataframe como CSV para su verificación manual
df.to_csv('dataset.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

# Configurar semilla para reproducibilidad
RANDOM_STATE = 42

# Split estratificado inicial: 80% train - 20% temporal
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=df['label']  # Estratificación por etiqueta
)

# Split estratificado del temporal: 10% dev - 10% test
dev_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=RANDOM_STATE,
    stratify=temp_df['label']  # Estratificación en el split secundario
)

# Resetear índices
train_df = train_df.reset_index(drop=True)
dev_df = dev_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Función para mostrar distribución de etiquetas
def print_distribution(df, name):
    dist = df['label'].value_counts(normalize=True).sort_index() * 100
    print(f"\nDistribución en {name}:")
    for label, percentage in dist.items():
        print(f"- {label}: {percentage:.1f}%")

# Verificar distribuciones
print(f"\nTamaños absolutos:")
print(f"Entrenamiento: {len(train_df)} ejemplos")
print(f"Desarrollo: {len(dev_df)} ejemplos")
print(f"Prueba: {len(test_df)} ejemplos")

print_distribution(df, 'dataset completo')
print_distribution(train_df, 'entrenamiento')
print_distribution(dev_df, 'desarrollo')
print_distribution(test_df, 'prueba')


Tamaños absolutos:
Entrenamiento: 3040 ejemplos
Desarrollo: 380 ejemplos
Prueba: 380 ejemplos

Distribución en dataset completo:
- A2: 10.5%
- B1_1: 23.3%
- B1_2: 49.4%
- B2: 16.8%

Distribución en entrenamiento:
- A2: 10.5%
- B1_1: 23.3%
- B1_2: 49.4%
- B2: 16.8%

Distribución en desarrollo:
- A2: 10.5%
- B1_1: 23.2%
- B1_2: 49.5%
- B2: 16.8%

Distribución en prueba:
- A2: 10.5%
- B1_1: 23.4%
- B1_2: 49.2%
- B2: 16.8%


In [ ]:
!pip install -U accelerate
!pip install --upgrade transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "valid": Dataset.from_pandas(dev_df.reset_index(drop=True)),
     "test": Dataset.from_pandas(test_df.reset_index(drop=True))
    }
    )
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'filename', 'label'],
        num_rows: 3040
    })
    valid: Dataset({
        features: ['content', 'filename', 'label'],
        num_rows: 380
    })
    test: Dataset({
        features: ['content', 'filename', 'label'],
        num_rows: 380
    })
})

In [ ]:
dataset['train'][122]

{'content': "If people have [***] it is – it is good – it is good preparation.  So if people have – but in case people or student can't study – study – study student – student don't do [***]...",
 'filename': 'SM_JPN_PTJ2_057_A2_0.txt',
 'label': 'A2'}

In [ ]:
dataset = dataset.class_encode_column("label")
labels = dataset['train'].features['label']
print(labels.names)

Casting to class labels:   0%|          | 0/3040 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/380 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/380 [00:00<?, ? examples/s]

['A2', 'B1_1', 'B1_2', 'B2']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

transformer_model = 'microsoft/deberta-v3-small'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = (AutoModelForSequenceClassification
         .from_pretrained(transformer_model, num_labels = len(labels.names))).to(device)
tokenizer=AutoTokenizer.from_pretrained(transformer_model)

def tokenize(batch):
    return tokenizer(batch["content"], truncation=False)

ds_enc = dataset.map(tokenize, batched=True)

ds_enc

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/3040 [00:00<?, ? examples/s]

Map:   0%|          | 0/380 [00:00<?, ? examples/s]

Map:   0%|          | 0/380 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'filename', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3040
    })
    valid: Dataset({
        features: ['content', 'filename', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 380
    })
    test: Dataset({
        features: ['content', 'filename', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 380
    })
})

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DebertaV2TokenizerFast(name_or_path='microsoft/deberta-v3-small', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=

In [ ]:
ds_enc['train'][0]

{'content': "I do not think that smoking should be completely banned to all the country because I disagree for various reasons.  First, people say that there isn't something good about smoking and it has only bad things like health problems or other problems but I think it's good for their psychological state.  I think they give smoking give relax to smokers and it – it gives them various mentally sense and I think it's good for them to feel free in the cho – choices of smoking problem.  Well, second it's bad for other people around them smokers because second smoking is – second hand smoking is very bad and it gives them [***] structure about other people.",
 'filename': 'SM_KOR_SMK1_077_B1_1.txt',
 'label': 1,
 'input_ids': [1,
  273,
  333,
  298,
  428,
  272,
  5614,
  403,
  282,
  1298,
  7648,
  264,
  305,
  262,
  658,
  401,
  273,
  8749,
  270,
  847,
  1725,
  260,
  1244,
  261,
  355,
  504,
  272,
  343,
  928,
  280,
  297,
  491,
  397,
  314,
  5614,
  263,
  278,
 

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 16
model_name = "CEFR-TextClassfinetuned_microsoft/deberta-v3-small"
training_args = TrainingArguments(
        output_dir=model_name,
        num_train_epochs=5,
        learning_rate = 2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_ratio=0.1,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model = "f1",
        load_best_model_at_end=True,
        save_total_limit = 1,
        report_to='none',
    )

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=ds_enc["train"],
                  eval_dataset=ds_enc["valid"],
                  tokenizer=tokenizer,
                 data_collator=data_collator,)
trainer.train()

<ipython-input-11-f73e18156329>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args,


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,1.210516,0.367878,0.476316,0.409552
2,No log,1.136380,0.355449,0.489474,0.411831
3,1.148500,1.149496,0.443117,0.471053,0.443074
4,1.148500,1.204509,0.453309,0.494737,0.464136
5,1.148500,1.220678,0.465651,0.497368,0.476914


TrainOutput(global_step=950, training_loss=0.9991386012027138, metrics={'train_runtime': 466.029, 'train_samples_per_second': 32.616, 'train_steps_per_second': 2.038, 'total_flos': 799096170946176.0, 'train_loss': 0.9991386012027138, 'epoch': 5.0})

In [ ]:
preds_output = trainer.predict(ds_enc['test'])
print(preds_output.metrics)

{'test_loss': 1.1640324592590332, 'test_precision': 0.4569869051791248, 'test_recall': 0.49736842105263157, 'test_f1': 0.47375113713133465, 'test_runtime': 3.8883, 'test_samples_per_second': 97.729, 'test_steps_per_second': 6.172}


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_true = np.array(ds_enc['test']["label"])
y_pred = preds_output.predictions.argmax(-1)

report = classification_report(
  y_true,
  y_pred,
  target_names=labels.names,
)
print(report)

              precision    recall  f1-score   support

          A2       0.00      0.00      0.00        40
        B1_1       0.40      0.56      0.47        89
        B1_2       0.61      0.60      0.61       187
          B2       0.38      0.41      0.39        64

    accuracy                           0.50       380
   macro avg       0.35      0.39      0.37       380
weighted avg       0.46      0.50      0.47       380



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
